## **DATASET**

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

ds = load_dataset("SKNahin/bengali-transliteration-data")

README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [3]:
ds['train']

Dataset({
    features: ['bn', 'rm'],
    num_rows: 5006
})

# **MODEL, TOKENIZER LOADING AND PREPROCESSING**

In [4]:
!pip install transformers

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [14]:
sentence1 = "I am a doctor"
sentence2 = "tumi kemon acho?"
tokenizer.src_lang = "en_XX"
encoded_hi = tokenizer(sentence2, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"]
)
output=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print("output",output)

output ['Tumi kemon acho?']


In [15]:
import pandas as pd

In [16]:
ds = ds['train'].train_test_split(test_size=0.1)
train_dataset = ds['train']
valid_dataset = ds['test']
ds

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 4505
    })
    test: Dataset({
        features: ['bn', 'rm'],
        num_rows: 501
    })
})

In [17]:
def preprocess_data(batch):
    inputs = tokenizer(batch['rm'], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(batch['bn'], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_dataset.map(preprocess_data, batched=True)
valid_dataset = valid_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/4505 [00:00<?, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

In [18]:
train_dataset

Dataset({
    features: ['bn', 'rm', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4505
})

# **FINE TUNING**

In [45]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    # predict_with_generate=True,
    #logging_dir="./logs",
    logging_steps=10
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [46]:
!nvidia-smi

Sat Dec 21 13:58:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             27W /   70W |    2445MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [47]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_params

128061440

In [48]:
def freeze_layers(model):
    
    for param in model.model.shared.parameters():
        param.requires_grad = False
    # for param in model.model.encoder.embed_tokens.parameters():
    #     param.requires_grad = False
    # for param in model.model.decoder.embed_tokens.parameters():
    #     param.requires_grad = False


    
    for i, layer in enumerate(model.model.encoder.layers):
       if i < 100:  
           for param in layer.final_layer_norm.parameters():
               param.requires_grad = False

    for i, layer in enumerate(model.model.decoder.layers):
       if i < 100:  
           for param in layer.final_layer_norm.parameters():
               param.requires_grad = False


    
    
    for i, layer in enumerate(model.model.encoder.layers):
        for param in layer.self_attn.parameters():
            param.requires_grad = False

    for i, layer in enumerate(model.model.decoder.layers):
        for param in layer.self_attn.parameters():
            param.requires_grad = False




    
    for i, layer in enumerate(model.model.encoder.layers):
        if i < 6:
            for param in layer.parameters():
                param.requires_grad = False
    for i, layer in enumerate(model.model.decoder.layers):
        if i < 6:
            for param in layer.parameters():
                param.requires_grad = False

In [49]:
freeze_layers(model)

In [50]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_params

128061440

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>